In [38]:
import pandas as pd
from openai import OpenAI


In [3]:
# df = pd.read_parquet('../Input/joined.parquet')
print(df.shape)
df.head()

(3000000, 13)


,Id,Title,Price,User_id,profileName,score,time,summary,text,authors,publisher,publishedDate,categories
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",4.0,1999-10-23,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...,[Julie Strain],None,1996,[Comics & Graphic Novels]
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,5.0,2004-09-21,Really Enjoyed It,I don't care much for Dr. Seuss but after read...,[Philip Nel],A&C Black,2005-01-01,[Biography & Autobiography]
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,5.0,2004-03-09,Essential for every personal and Public Library,"If people become the books they read and if ""t...",[Philip Nel],A&C Black,2005-01-01,[Biography & Autobiography]
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",4.0,2004-07-25,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",[Philip Nel],A&C Black,2005-01-01,[Biography & Autobiography]
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",4.0,2005-02-10,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...,[Philip Nel],A&C Black,2005-01-01,[Biography & Autobiography]


In [18]:
df_authors = df.explode('authors')
grouped_authors_df = df_authors.groupby('authors').agg(
    count=('score', 'size'),
    mean_score=('score', 'mean'),
    median_score=('score', 'median'),
    variance_score=('score', 'var')
).reset_index()

In [19]:
grouped_authors_df.sort_values(by=['mean_score', 'count'], ascending=False)

,authors,count,mean_score,median_score,variance_score
102335,Michelle Cole,113,5.0,5.0,0.0
117854,Randall Jarrell,74,5.0,5.0,0.0
31412,David Seabury,52,5.0,5.0,0.0
14211,Blaine Harris,38,5.0,5.0,0.0
20399,Charles A. Coonradt,38,5.0,5.0,0.0
...,...,...,...,...,...
151968,Yehudi O. Webster,1,1.0,1.0,NaN
152061,Yogeshwari Phatak,1,1.0,1.0,NaN
152129,York Membery,1,1.0,1.0,NaN
152167,Yoshiyasu Takefuji,1,1.0,1.0,NaN


In [22]:
grouped_authors_df[grouped_authors_df['count']>8].sort_values(by=['variance_score'], ascending=False)

,authors,count,mean_score,median_score,variance_score
122467,Robert Boyer,9,3.222222,5.0,4.444444
23990,Cinda Voegtli,10,3.000000,3.0,4.444444
108791,Orion Moshe Kopelman,10,3.000000,3.0,4.444444
1060,Adam Josephs,10,3.000000,3.0,4.444444
95293,Mark Manfield,9,3.222222,5.0,4.444444
...,...,...,...,...,...
19856,Catholic Church. Pope (1963-1978 : Paul VI),12,5.000000,5.0,0.000000
7867,Annie B. Bond,16,5.000000,5.0,0.000000
118776,Rebecca R. Springer,11,5.000000,5.0,0.000000
118790,Rebecca Ruter Springer,9,5.000000,5.0,0.000000


In [26]:
df_authors['text'] = df_authors['text'].str[:50]

In [27]:
df_authors.head().to_clipboard()

In [32]:
def get_reviews_by_author(df, author_name):
    # Filter the dataframe for the given author
    author_reviews = df[df['authors'] == author_name]
    
    # Create a dictionary to store one review per score
    reviews_dict = {}
    
    for _, row in author_reviews.iterrows():
        score = row['score']
        if score not in reviews_dict:
            reviews_dict[score] = {
                'Title': row['Title'],
                'Text': row['text'],
                'Score': score
            }
    
    return reviews_dict

In [35]:
oi = get_reviews_by_author(df_authors, 'Robert Boyer')

In [49]:
client = OpenAI()
def analyze_author_reviews(author_name, reviews_dict):
    # Prepare the reviews text to be included in the message
    reviews_text = "\n".join(
        [f"Title: {review['Title']}\nReview: {review['Text']}\n"
         for review in reviews_dict.values()]
    )
    
    # Create the system message
    system_message = (
        f"You are an Editor and our comppany is considering publishing this author {author_name}, and we need a thorough evaluation. "
        "Based on the following reviews, please provide an analysis of the author's strengths and weaknesses. "
        
    )
    
    # Send the request to OpenAI
    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": reviews_text}
        ]
    )
    
    # Extract and return the response
    return response.choices[0].message.content

In [50]:

# Analyze the author based on the reviews
analysis_result = analyze_author_reviews('Robert Boyer' ,oi )
print(analysis_result)

ChatCompletion(id='chatcmpl-9sGv7nz2e9rB1QxEKcqV3gvOagKJH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Robert Boyer: Author Analysis Based on Reviews**\n\n**Strengths:**\n\n1. **Comprehensive Coverage**: Robert Boyer\'s book title, "The Amazing Art of Pyrography," suggests an extensive exploration of the subject. The reviews imply that readers looking for a thorough guide will find this book useful.\n\n2. **Depth of Knowledge**: Given that the book is described as providing a complete experience, it indicates Boyer\'s deep knowledge of pyrography, making him a reliable source for enthusiasts and professionals alike.\n\n3. **Market Positioning**: His work is positioned as a must-have for serious readers in the niche of pyrography. This suggests a strong value proposition and relevance to the target audience, enhancing its marketability.\n\n**Weaknesses:**\n\n1. **Review Incompleteness**: The reviews provided are incomplete and 